In [1]:
import numpy as np 
import pandas as pd
import re

## Exercises

**1. Write a function named is_vowel. It should accept a string as input and use a regular expression to determine if the passed string is a vowel. While not explicity mentioned in the lesson, you can treat the result of re.search as a boolean value that indicates whether or not the regular expression matches the given string.**



In [2]:
def is_vowel(string):
    regexp = r'[aeiouAEIOU]'
    return bool(re.search(regexp, string))

In [3]:
#Test function
print('a:', is_vowel('a'))
print('t:', is_vowel('t'))
print('E:', is_vowel('E'))

a: True
t: False
E: True


**2. Write a function named is_valid_username that accepts a string as input. A valid username starts with a lowercase letter, and only consists of lowercase letters, numbers, or the _ character. It should also be no longer than 32 characters. The function should return either True or False depending on whether the passed string is a valid username.**

In [4]:
def is_valid_username(string):
    regexp = r'^[a-z][a-zA-Z0-9_]{,31}$'
    return bool(re.search(regexp, string))

In [5]:
#Test function 
print('aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa:', is_valid_username('aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'))
print('codeup:', is_valid_username('codeup'))
print('Codeup:', is_valid_username('Codeup'))
print('codeup123:', is_valid_username('codeup123'))
print('1codeup:', is_valid_username('1codeup'))

aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa: False
codeup: True
Codeup: False
codeup123: True
1codeup: False


**3. Write a regular expression to capture phone numbers. It should match all of the following:**
(210) 867 5309
, +1 210.867.5309
, 867-5309
, 210-867-5309



In [6]:
regexp = r'''
(?P<country_code>\+\d+)?
\D*
(?P<area_code>\d{3})?
\D*
(?P<exchange_code>\d{3})
\D*
(?P<last_four>\d{4})$
'''

numbers = pd.Series([
    '(210) 867 5309',
    '+1 210.867.5309',
    '867-5309',
    '210-867-5309',
], name='original')

pd.concat([numbers, numbers.str.extract(regexp, re.VERBOSE)], axis=1)

,original,country_code,area_code,exchange_code,last_four
0,(210) 867 5309,NaN,210,867,5309
1,+1 210.867.5309,+1,210,867,5309
2,867-5309,NaN,NaN,867,5309
3,210-867-5309,NaN,210,867,5309


**4. Use regular expressions to convert the dates below to the standardized year-month-day format.**
- 02/04/19
- 02/05/19
- 02/06/19
- 02/07/19
- 02/08/19
- 02/09/19
- 02/10/19


In [7]:
dates = pd.Series([
    '02/04/19',
    '02/05/19',
    '02/06/19',
    '02/07/19',
    '02/08/19',
    '02/09/19',
    '02/10/19',
])
dates.str.replace(r'(\d+)/(\d+)/(\d+)', r'20\3-\1-\2', regex=True)

0    2019-02-04
1    2019-02-05
2    2019-02-06
3    2019-02-07
4    2019-02-08
5    2019-02-09
6    2019-02-10
dtype: object

**5. Write a regex to extract the various parts of these logfile lines:**
    
GET /api/v1/sales?page=86 [16/Apr/2019:193452+0000] HTTP/1.1 {200} 510348 "python-requests/2.21.0" 97.105.19.58
POST /users_accounts/file-upload [16/Apr/2019:193452+0000] HTTP/1.1 {201} 42 "User-Agent: Mozilla/5.0 (X11; Fedora; Fedora; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36" 97.105.19.58
GET /api/v1/items?page=3 [16/Apr/2019:193453+0000] HTTP/1.1 {429} 3561 "python-requests/2.21.0" 97.105.19.58


In [8]:
logfiles = """GET /api/v1/sales?page=86 [16/Apr/2019:193452+0000] HTTP/1.1 {200} 510348 "python-requests/2.21.0" 97.105.19.58
POST /users_accounts/file-upload [16/Apr/2019:193452+0000] HTTP/1.1 {201} 42 "User-Agent: Mozilla/5.0 (X11; Fedora; Fedora; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36" 97.105.19.58
GET /api/v1/items?page=3 [16/Apr/2019:193453+0000] HTTP/1.1 {429} 3561 "python-requests/2.21.0" 97.105.19.58"""

In [10]:
#Create a df with the original logfiles
df = pd.DataFrame()
#Create an original columns
df['original'] = logfiles.strip().split('\n')
df

,original
0,GET /api/v1/sales?page=86 [16/Apr/2019:193452+...
1,POST /users_accounts/file-upload [16/Apr/2019:...
2,GET /api/v1/items?page=3 [16/Apr/2019:193453+0...


In [11]:
# Write the regexp
regexp = r'''
^(?P<method>GET|POST)
\s+
(?P<path>.*?)
\s+
\[(?P<timestamp>.*?)\]
\s+
(?P<http_version>.*?)
\s+
\{(?P<status>\d+)\}
\s+
(?P<bytes_sent>\d+)
\s+
"(?P<user_agent>.+)"
\s
(?P<ip>.+)
'''

# concat the capture groups onto my df
df = pd.concat([df, df.original.str.extract(regexp, re.VERBOSE)], axis = 1)

In [12]:
df

,original,method,path,timestamp,http_version,status,bytes_sent,user_agent,ip
0,GET /api/v1/sales?page=86 [16/Apr/2019:193452+...,GET,/api/v1/sales?page=86,16/Apr/2019:193452+0000,HTTP/1.1,200,510348,python-requests/2.21.0,97.105.19.58
1,POST /users_accounts/file-upload [16/Apr/2019:...,POST,/users_accounts/file-upload,16/Apr/2019:193452+0000,HTTP/1.1,201,42,User-Agent: Mozilla/5.0 (X11; Fedora; Fedora; ...,97.105.19.58
2,GET /api/v1/items?page=3 [16/Apr/2019:193453+0...,GET,/api/v1/items?page=3,16/Apr/2019:193453+0000,HTTP/1.1,429,3561,python-requests/2.21.0,97.105.19.58
